# Unit 2 Lesson 7 - Joint 2D and AngryAvians
In this lesson, we'll learn about new Unity physics components called Joint2D and use everything we learned so far to create AngryAvians(Angry Birds clone).

---

# Learning Objectives
---

*   Students will learn about Joint2D
*   Students will apply previous lesson to create an Angry Birds clone

# Key Concepts
---

*   Joint2D, HingeJoint2D, SpringJoint2D
*   Inheritance
*   Creating Angry Avians

# Introduction
---

Another useful physics components that Unity provide are the varieties of Joint2D. Joint2D allows for GameObjects to attach together based on different joints behavior. While, there are a lot of Joint2D to explore, for this Angry Birds clone, we will only use HingeJoint2D. It is recommended to checkout the 2D Joints documentation to learn more about the components.

Unity Documentation: https://docs.unity3d.com/Manual/Joints2D.html

For this project, we will use bit and pieces of concepts that we learned so far from previous lessons to build out an Angry Birds clone, Angry Avians!

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_1.png" alt="Drawing"/>

All of the sprites used in this lesson will be provided.

# Concept 1: HingeJoint2D, SpringJoint2D

---
### **HingeJoint2D**

#### What is HingeJoint2D? 
From the Unity Documentation, HingeJoint2D allows GameObject to be attached to a point in space which it can rotate around. 

Documentation: https://docs.unity3d.com/Manual/class-HingeJoint2D.html

#### How to use HingeJoint2D?

To use HingeJoint2D, we need to have in mind what object we want to rotate around from a point. In this project, there are two instances of Joints
that were used. First, let's look at how to use hinge joint to create a beam

1. Create a game object using the beam sprite. Attach rigidbody2d and collider.
2. Attach HingeJoint2D. We can also change the anchor location where the beam is set to rotate around.

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_2.png" alt="Drawing"/>

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_1.gif" alt="Drawing"/>



## Walkthrough: Adding birds and SpringJoints
---

### What is SpringJoint2D? 
SpringJoint2D allows for 2 game objects to attach to each other by a "spring". This is good for anything that required stretchiness, like bird and slingshot. 

Documentation: https://docs.unity3d.com/Manual/class-SpringJoint2D.html

#### Creating Slingshot

Let's now apply SpringJoint to our bird and use our mouse to drag the bird around.

1. Create the slingshot for bird anchor location. First, create the Slingshot game object with the following structure:

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_3.png" alt="Drawing"/>

The parent Slingshot game object, will have the front sprite of the slingshot, with a rigidbody2d set to static.

The backsprite will use the backsprite of the slingshot.

Anchor will be the center between slingshot and backsprite.

2. Next, let's set up the bird anchor onto the slingshot. First, create the game object with the bird sprite, and attach rigidbody, and collider.

3. Then, attach SpringJoint2D component to the bird. 

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_4.png" alt="Drawing"/>

We want "Connected Rigid Body" to be attach to the slingshot game object that we created previously.

Next, we want to move our bird closer to the anchor, where "distance" is close to 0

Lastly, we want to "Frequency" to be aroudn 1.5.

4. Let's create scripts to control move the bird, and shoot the bird from the slingshot. First, create a script called "Slingshot" and attach to our slingshot game object:

In [ ]:
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using UnityEngine.SceneManagement;
using UnityEngine.UIElements;

public class Slingshot : MonoBehaviour
{
    private int index = 0;
    [SerializeField]
    private BirdController[] birds;
    public BirdController currentBird;
    public BirdController releasedBird;

    [SerializeField] private float MAX_DISTANCE;

    [SerializeField]
    private Transform anchor;

    [SerializeField]
    private float releaseTime;

    private bool isMouseDown;
    public bool HoldBird { get; private set;  }
    
    private Rigidbody2D rb;
    private SpringJoint2D springJoint;

    // Start is called before the first frame update
    void Start()
    {
        currentBird = birds[index++];
        rb = currentBird.RB;
        springJoint = currentBird.SpringJoint;
    }

    // Update is called once per frame
    void Update()
    {

        if (Input.GetKeyDown(KeyCode.R))
        {
            SceneManager.LoadScene(SceneManager.GetActiveScene().name);
        }

        if (Input.GetMouseButtonDown((int)MouseButton.LeftMouse))
        {
            isMouseDown = true;
            releasedBird = null; // kinda jank // this stops camera to interp to release bird
        }

        if (Input.GetMouseButtonUp((int)MouseButton.LeftMouse))
        {
            rb.isKinematic = false;
            isMouseDown = false;

            // Release
            if (HoldBird)
            {
                StartCoroutine(Release(releaseTime));
            }
            HoldBird = false;
        }

        if (isMouseDown && HoldBird)
        {
            Vector2 mouseWorldPos = Camera.main.ScreenToWorldPoint(Input.mousePosition);
            Vector2 connectedAnchorWorldPos = anchor.position;// transform.TransformPoint(springJoint.connectedAnchor);
            Debug.Log(connectedAnchorWorldPos);
            Vector2 mouseDir = (mouseWorldPos - connectedAnchorWorldPos).normalized;
            float mouseLength = Mathf.Abs(Vector2.Distance(mouseWorldPos, connectedAnchorWorldPos));//springJoint.distance;//

            Vector2 movePoint = mouseLength <= MAX_DISTANCE ?
                                mouseWorldPos :
                                connectedAnchorWorldPos + mouseDir * MAX_DISTANCE;

            rb.MovePosition(movePoint);
            rb.isKinematic = true;
        }

    }

    private void FixedUpdate()
    {
        RaycastHit2D rayHit;

        if (isMouseDown)
        {
            rayHit = Physics2D.Raycast(Camera.main.ScreenToWorldPoint(Input.mousePosition), Vector3.back);
            if (rayHit.collider != null)
            {
                if (rayHit.collider.CompareTag("Bird"))
                {
                    HoldBird = true;
                }
            }
        }
    }

    private IEnumerator Release(float time)
    {
        currentBird.Release();
        yield return new WaitForSeconds(time);
        springJoint.enabled = false;
        releasedBird = currentBird;

        yield return new WaitForSeconds(1f);
        if (index < birds.Length)
        {
            currentBird = birds[index++];
            currentBird.transform.position = anchor.position;
            rb = currentBird.RB;
            springJoint = currentBird.SpringJoint;
        }
    }

}


#### Creating our Avian
Next, we can to create a script for our bird and attach it to the game object. I named my script to be "BirdController". Also, we want to create a "Bird" tag for our bird.

In [ ]:
using System;
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using UnityEngine.UIElements;

using UnityEngine.SceneManagement;

public class BirdController : MonoBehaviour
{
    protected SpriteRenderer sr;

    public Rigidbody2D RB { get; private set; }
    public SpringJoint2D SpringJoint { get; private set; }

    public const string DIRECTORY = "Sprites/Avians/DefaultAvian/";
    [SerializeField] protected Sprite defaultSprite;
    [SerializeField] protected Sprite flyingSprite;

    protected bool isFlying;


    // Start is called before the first frame update
    void Awake()
    {
        RB = GetComponent<Rigidbody2D>();
        SpringJoint = GetComponent<SpringJoint2D>();
        //anchor = springJoint.attachedRigidbody.transform.GetChild(0);

        Debug.Log(defaultSprite);
        Debug.Log(flyingSprite);
    }

    protected virtual void Start()
    {
        isFlying = false;
        SetSprite(DIRECTORY);
    }

    protected void SetSprite(String dir)
    {
        sr = this.transform.GetChild(0).GetComponentInChildren<SpriteRenderer>();
        defaultSprite = sr.sprite;
        flyingSprite = Resources.Load<Sprite>(dir + "avian_fly") as Sprite;
    }

    public void Release()
    {
        sr.sprite = flyingSprite;
        isFlying = true;
    }
}


Now, go back to our slingshot component, we want to attach our birdcontroller to the array of birds. Drag in the anchor location to the anchor option. Change max distance to 5 or some appropriate value, and release time to 0.2. These values are subject to change if needed.
<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_5.png" alt="Drawing"/>

If everything set up correctly, we should be able to do this:
<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_2.gif" alt="Drawing"/>


There are a few small things that we set up that does not match up with the gif such as the animation when flying and bird bobing. This is extra animation added in the game. We will go back to this in later activities. 

Recap on what we did in this first part. We created a basic slingshot system where we can drag the bird and release it through the help of coroutine, and springjoint. In the next concept, we will go over how to add more special bird using a new programming concept of inheritance.


# Concept 2: Inheritance
---
#### What is Inheritance? 

In programming, inheritance is a concept of inherit or basing a new object on an existing one. In other word, we creating a new object by derive it from an existing one, and add in or overide old functionalities. This is a very powerful concept in object-oriented programming as it is the meat and bone of how a lot of programs built. 

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_6.png" alt="Drawing"/>

#### Inheritance for Angry Avians

With that said, how can we use this concept for our game? One idea is to create different type of avians that based on the original bird- an egg bird that shooting out an exploding egg!



Let's use inheritance to create this bird

1. First we need to create the prefab game object for this bird. A quick way to do this is to clone the angrybird prefab, and then change the sprite.

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_7.png" alt="Drawing"/>

2. Next, we need to replace the original BirdController to a derived version with new special functionality for shooting egg, and still retain the original functions. Let's create a new script called "EggBirdController"

In [ ]:
using System;
using System.Collections;
using System.Collections.Generic;
using UnityEngine;
using UnityEngine.UIElements;

using UnityEngine.SceneManagement;

public class EggBirdController : BirdController
{
    public new const string DIRECTORY = "Sprites/Avians/EggAvian/";
    [SerializeField] private float EGG_FORCE = 5f;
    [SerializeField] private GameObject eggGO;
    [SerializeField] private float eggCount = 1f;

    private float EGG_DISTANCE = 5f;

    protected override void Start()
    {
        base.Start();
        SetSprite(DIRECTORY);
    }

    // Update is called once per frame
    void Update()
    {
        if (Input.GetMouseButtonDown(0) && isFlying && eggCount-- > 0)
        {
            Vector3 direction = Camera.main.ScreenToWorldPoint(Input.mousePosition) - transform.position;
            Egg egg = Instantiate(eggGO, transform.position + direction.normalized * EGG_DISTANCE, Quaternion.identity).GetComponent<Egg>();
            egg.Push(EGG_FORCE * RB.mass * direction, ForceMode2D.Impulse);
            RB.AddForce(EGG_FORCE * RB.mass * -direction, ForceMode2D.Impulse);
        }
    }
}


Once that is done, replace it in the inspector tab:
<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_8.png" alt="Drawing"/>

3. 
You noticed in the script and the inspector that there is an option for EggGO, this will be the prefab game object for our exploding egg. Let's create that prefab

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_9.png" alt="Drawing"/>

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_10.png" alt="Drawing"/>

Next, let's add the script Egg for this game object to control the explosion:

In [ ]:
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class Egg : MonoBehaviour
{
    [SerializeField] private Rigidbody2D rb;
    [SerializeField] private const float MOVEMENT_THRESHHOLD = 0.1f;
    [SerializeField] private const float DESTROY_WAIT = 1f;
    [SerializeField] private GameObject explosionArea;
    // Start is called before the first frame update
    void Awake()
    {
        rb = GetComponent<Rigidbody2D>();
    }

    public void Push(Vector2 vec, ForceMode2D mode)
    {
        rb.AddForce(vec, mode);
    }

    // Update is called once per frame
    void Update()
    {
        if (rb.velocity.magnitude < MOVEMENT_THRESHHOLD)
        {
            Destroy(this.gameObject, DESTROY_WAIT);
        }
    }

    private void OnCollisionEnter2D(Collision2D collision)
    {
        GameObject explosion = Instantiate(explosionArea, this.transform.position, Quaternion.identity);
        Destroy(this.gameObject);
        Destroy(explosion, 0.1f);
    }

}


There is still one missing thing. We need to define an exploding area. Recalled that we already learned this in the previous lesson with Effector. Point Effector would be a great way to define an exploding area!

4. Let's create the exploding prefab: 
<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_11.png" alt="Drawing"/>

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_12.png" alt="Drawing"/>

With the exploding prefab, drag it into EggBirdController script.

5. Once that is done, we now have everything to put together the new avian type! In your slingshot gameobject, drag our eggbird prefab into the Birds array: 

<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_13.png" alt="Drawing"/>

6. Let's see the new bird in action:
<img src="https://raw.githubusercontent.com/jcortezzo/TCS-GameDev-Curriculum/master/Unit%202/Images/U2L7_14.gif.png" alt="Drawing"/>

---
## Activity


---
## Activity #1

---

**Problem:** Add bird idle and flying animation

##### **Solution**


---
## Activity #2

---

**Problem:** Adding a new bird type that have extra dash while in the air

##### **Solution**


 

# HOMEWORK
---

## HW #1

---

**Problem:** 

##### **Solution**

